#  Lab 2: Stability and Multistep Methods
Gwen Lofman

In [ ]:
(require '[clojupyter.misc.helper :as helper])
(require '[clojupyter.misc.display :as display])

(helper/add-dependencies '[incanter "1.5.7"])
(helper/add-dependencies '[org.clojure/math.numeric-tower "0.0.4"])

(require '[clojure.core.reducers :as r])
(require '[clojure.math.numeric-tower :as math])
(use '(incanter core stats charts io))

## Stability of Forward and Backward Euler

## 4th order explicit Runge Kutta Method

In [ ]:
(defn rk4
  "Runge-Kutta explicit 4th order method

  Use the function `f` which takes two variables, y and t, its
  initial values `t0` and `y0` and solve the initial value
  problem, stopping at `tf`, taking steps of size `dt`.

  Convergence is O(`dt`^4), and this method is not A-Stable.

  Returns the vector `ys` of approximate solutions to the
  initial value problem, along with the vector `ts` of t values
  associated with each y in the form [`ys` `ts`]."
  [f y0 t0 tf dt]
  (loop [yn y0 ys [y0] tn t0 ts [t0]]
    (if (< tn tf)
      (let [;; Calculate k stage-by-stage for efficiency
            k1 (f yn tn)
            k2 (f (+ yn (* dt 0.5 k1)) (+ tn (* dt 0.5)))
            k3 (f (+ yn (* dt 0.5 k2)) (+ tn (* dt 0.5)))
            k4 (f (+ yn (* dt k3)) (+ tn dt))
            ;; Calculate the new yn from the stages
            ks (+ (* k1 (/ 1. 6))
                  (* k2 (/ 1. 3))
                  (* k3 (/ 1. 3))
                  (* k4 (/ 1. 6)))
            yn (+ yn (* dt ks))
            tn (+ tn dt)]
        (recur yn (conj ys yn) tn (conj ts tn)))
      [ys ts])))

We can verify our implementation using a function with a known exact solution.  To do this we will use:

$$
y(t) = y(t) * \sin(t)
$$

where the exact solution is

$$
y(t) = -e^{1 - \cos(t)}
$$

In [ ]:
(defn y [yn tn] (* yn (Math/sin tn)))

(defn y-exact [t]
  (- (Math/pow Math/E (- 1 (Math/cos t)))))

For each $\Delta t$ in $[\frac{1}{4}, \frac{1}{8}, \frac{1}{16}, \frac{1}{32} \frac{1}{64}]$, we can plot the results and compare the global truncation error at $t = 1$.

In [ ]:
(def dts [(/ 1. 4) (/ 1. 8) (/ 1. 16) (/ 1. 32) (/ 1. 64)])

In [ ]:
(defn compare-rk4
  "Compare Runge-Kutta 4th order explicit method for given dts"
  [dts]
  (reduce
   #(let [s (str "dt of " %2)
          [ys ts] (rk4 y -1 0 1 %2)]
      (add-lines %1 ts ys :series-label s))
   (function-plot y-exact 0 1 :title "y(t) vs rk4" :legend true)
   dts))

In [ ]:
(defn plot-truncation-err
  "Plot the error for each `dt` in `dts` at `t = 1` in loglog"
  [test-function exact dts]
  (let [es  (mapv (comp #(Math/log10 %) #(Math/abs (- exact %))
                        last first test-function) dts)
        dts (mapv #(Math/log10 %) dts)]
    (doto (xy-plot dts es
                   :title "Error at t=1"
                   :x-label "log10(dt)"
                   :y-label "log10(error)")
      (set-y-range (reduce min es) (reduce max es)))))

First visually plotting the results just to see how it performs globally:

In [ ]:
(-> (compare-rk4 dts)
    (set-y-range -1.625 -0.95)
    (.createBufferedImage 600 300))

And then plotting the global truncation error at $t = 1$ in loglog scale to see the 4th order convergence:

In [ ]:
(-> (plot-truncation-err #(rk4 y -1 0 1 %) (y-exact 1) dts)
    (.createBufferedImage 600 400))

Remember, because this is in loglog scale, step-size increases to the right and error increases upwards.